# Heuristics - CVRP

In [24]:
from itertools import cycle

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import functions as f
import importlib

## Instantiate data

In [25]:
dataset = pd.read_csv("./data/custom_19.csv", index_col=0)
coordinates = dataset.loc[:, ["x", "y"]]

In [26]:
n_vehicles = 4
start_locations = [0, 2, 3, 5]
end_locations = [0, 2, 3, 5]

allow_arbitrary_end_locations = True  # when True, the end locations are not fixed
# this is done by adding a dummy node to the graph that has zero travel time to all locations

if allow_arbitrary_end_locations:
    end_locations = [len(coordinates)] * n_vehicles
    arbitrary_end_location = pd.DataFrame(
        {"x": {len(coordinates): 99}, "y": {len(coordinates): 99}}
    )
    coordinates = pd.concat([coordinates, arbitrary_end_location], axis=0)

N = coordinates.shape[0]

In [27]:
# assume travel time is proportional to euclidean distance
time_matrix = squareform(pdist(coordinates, metric="euclidean"))
time_matrix = np.round(time_matrix * 1e4, decimals=0).astype(int)

if allow_arbitrary_end_locations:
    time_matrix[:, -1] = time_matrix[-1, :] = 0

## Model

In [28]:
# Create the routing index manager: number of nodes, number of vehicles, depot node
manager = pywrapcp.RoutingIndexManager(N, n_vehicles, start_locations, end_locations)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

### Parameters

In [29]:
# Same valid for any callback related to arcs/edges
def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return time_matrix[from_node, to_node]


transit_callback_index = routing.RegisterTransitCallback(time_callback)

In [30]:
# # Same valid for any callback related to nodes
# def demand_callback(from_index):
#     from_node = manager.IndexToNode(from_index)
#     return demands[from_node]

# demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

### Constraints

In [31]:
# # Any constraint associated with vehicles can take same arguments
# routing.AddDimensionWithVehicleCapacity(
#     demand_callback_index,
#     0,  # null capacity slack
#     [capacity,] * n_vehicles,  # vehicle maximum capacities (list for each vehicle)
#     True,  # start cumul to zero
#     'Capacity'
# )

### Objective

In [32]:
# Define cost of each arc
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

### Solution

In [33]:
# Setting heuristic strategies
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.time_limit.FromSeconds(1)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

In [34]:
solution.ObjectiveValue()

6766935

### Plot Results

In [35]:
importlib.reload(f)
tours = f.compile_tours(
    n_vehicles, routing, manager, solution, allow_arbitrary_end_locations
)

In [36]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()